In [2]:
import praw
from collections import Counter
import pandas as pd
import psaw
import psycopg2
from  psaw import PushshiftAPI
from datetime import datetime
import pickle
import nltk

In [3]:
api = PushshiftAPI()

In [4]:
reddit = praw.Reddit(
    client_id="MtjmLCtDmwlKT8ryL_Mgog",
    client_secret="E2RKMDZtQvR9HSRGGOw-wnU7bsqZ0Q",
    password="Jch4494!!",
    user_agent="testscript by u/94_meowmeow",
    username="94_meowmeow",
)

In [5]:
def replies_of(top_level_comment, count_comment, sub_entries_textblob, sub_entries_nltk):
    if len(top_level_comment.replies) == 0:
        count_comment = 0
        return
    else:
        for num, comment in enumerate(top_level_comment.replies):
            try:
                count_comment += 1
                #print('-' * count_comment, comment.body)
                text_blob_sentiment(comment.body, sub_entries_textblob)
                nltk_sentiment(comment.body, sub_entries_nltk)
            except:
                continue
            replies_of(comment, count_comment, sub_entries_textblob,sub_entries_nltk)

            #https://towardsdatascience.com/automate-sentiment-analysis-process-for-reddit-post-textblob-and-vader-8a79c269522f

In [6]:
def replies_of_content(top_level_comment, count_comment):
    if len(top_level_comment.replies) == 0:
        count_comment = 0
        return
    else:
        for num, comment in enumerate(top_level_comment.replies):
            try:
                count_comment += 1
                #print('-' * count_comment, comment.body)
            except:
                continue
            replies_of(comment, count_comment, sub_entries_textblob,sub_entries_nltk)

In [4]:
def wsb_scrape_all(start_epoch, end_epoch, limit_size):
    #all_posts=[]
    soup=[]
    top_posts = list(api.search_submissions(after=start_epoch,before=end_epoch, subreddit='wallstreetbets', sort = 'desc', sort_type = 'score', limit=limit_size))
    for submission in list(top_posts):
            submission_comm = reddit.submission(id=submission.id)
            #general_data = [submission.title, dt.datetime.fromtimestamp(submission.created_utc), submission.url]
            #all_posts.append(general_data)
            for count, top_level_comment in enumerate(submission_comm.comments):
                count_comm = 0
                try :
                    soup.append((top_level_comment.body, dt.datetime.fromtimestamp(submission.created_utc)))
                    replies_of_content(top_level_comment,
                               count_comm)
                except:
                    continue    

    return(soup)

In [7]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
sia = SentimentIntensityAnalyzer()
def nltk_sentiment(review, sub_entries_nltk):
    vs = sia.polarity_scores(review)
    if not vs['neg'] > 0.05:
        if vs['pos'] - vs['neg'] > 0:
            sub_entries_nltk['positive']=sub_entries_nltk['positive'] + 1
            return 'Positive'
        else:
            sub_entries_nltk['neutral']=sub_entries_nltk['neutral'] + 1
            return 'Neutral'
    elif not vs['pos'] > 0.05:
        if vs['pos'] - vs['neg'] <= 0:
            sub_entries_nltk['negative']=sub_entries_nltk['negative'] + 1
            return 'Negative'
        else:
            sub_entries_nltk['neutral']=sub_entries_nltk['neutral'] + 1
            return 'Neutral'
    else:
        sub_entries_nltk['neutral']=sub_entries_nltk['neutral'] + 1
        return 'Neutral'

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jenniferhilibrand/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
def wsb_scrape_titles(start_epoch, end_epoch, limit_size):
    overall_data=[]
    top_posts = list(api.search_submissions(after=start_epoch,before=end_epoch, subreddit='wallstreetbets', sort = 'desc', sort_type = 'score', limit=limit_size))
    for submission in list(top_posts):
            general_data = [submission.title,dt.datetime.fromtimestamp(submission.created_utc), submission.url]
            words = submission.title.split()
            title_tickers=[]

            content_tickers=[]
            sub_entries_nltk = {'negative': 0, 'positive' : 0, 'neutral' : 0}

            nltk_sentiment(submission.title, sub_entries_nltk)
            submission_comm = reddit.submission(id=submission.id)
            #timestamp = reddit.submission()
            for count, top_level_comment in enumerate(submission_comm.comments):

                count_comm = 0
                try :

                    nltk_sentiment(top_level_comment.body, sub_entries_nltk)
                    replies_of(top_level_comment,
                               count_comm,
                               sub_entries_textblob,
                               sub_entries_nltk)
                    for i in top_level_comment.body.split():
                        content_tickers.extend(extract_ticker(i))
                except:
                    continue
            total_comments = sub_entries_nltk['negative']+sub_entries_nltk['neutral']+sub_entries_nltk['positive']


            vader_pos_perc = sub_entries_nltk['positive']/total_comments
            vader_neg_perc = sub_entries_nltk['negative']/total_comments

            sentiment_data=[]
            sentiment_data = [submission.title, sub_entries_nltk, total_comments, vader_neg_perc, vader_pos_perc, title_tickers, dt.datetime.fromtimestamp(submission.created_utc), submission.url]


            overall_data.append(sentiment_data)
                
    
    return(overall_data)

In [11]:
import datetime as dt
start_date=dt.date(2021,6,1) 
end_date = dt.date(2021,6,30)
#soup_df3= pd.DataFrame(wsb_scrape_all(start_date, end_date, 100))


'''
with open('/Users/jenniferhilibrand/Metis/NLP/WSB_Jun_Soup.pickle', 'wb') as to_save:
    pickle.dump(soup_df3 ,to_save)'''

"\nwith open('/Users/jenniferhilibrand/Metis/NLP/WSB_Jun_Soup.pickle', 'wb') as to_save:\n    pickle.dump(soup_df3 ,to_save)"

In [22]:
import datetime as dt
start_date=dt.date(2021,1,1) 
end_date = dt.date(2021,6,30)
submissions = list(api.search_submissions(after=start_date, before= end_date,
                            subreddit='wallstreetbets', sort = 'desc', sort_type = 'score',
                            filter=['url','author', 'title', 'subreddit'],
                            limit=2000))

titles=[]
times=[]
for submission in submissions: 
    times.append(dt.datetime.fromtimestamp(submission.created_utc))
    titles.append(submission.title)

titles_df = pd.DataFrame(titles)
titles_df['timestamp']=times

from datetime import datetime
cols = ['text','timestamp']
titles_df.columns = cols
dates=[]
for i in titles_df['timestamp']:
    dates.append(dt.datetime.date(i))
titles_df['dates']=dates

titles_df.head(10)

,text,timestamp,dates
0,COULDN’T POSSIBLY BE HEDGE FUNDS 💎👐💎,2021-02-03 14:54:00,2021-02-03
1,My 102 year old grandmother bought GME in Augu...,2021-02-02 08:29:24,2021-02-02
2,who else bought at $250+,2021-02-04 12:56:48,2021-02-04
3,Who of my 300 brethren's are still in ?,2021-02-03 03:54:46,2021-02-03
4,APES IN CONTROL,2021-02-02 13:01:05,2021-02-02
5,Billionaire Investor Mark Cuban Says Reddit St...,2021-02-01 14:03:19,2021-02-01
6,Me too,2021-01-31 20:11:26,2021-01-31
7,Together no one can stop us !!!,2021-02-02 21:14:54,2021-02-02
8,Spread the word! 💎🤲,2021-01-31 22:14:44,2021-01-31
9,"If this isn't a sign, I don't know what is...🚀...",2021-02-02 18:33:03,2021-02-02


In [25]:
with open('/Users/jenniferhilibrand/Metis/NLP/WSB_titles_only.pickle', 'wb') as to_save:
    pickle.dump(titles_df ,to_save)

In [ ]:
titles_soup = pd.DataFrame(wsb_scrape_titles(start_date, end_date, 2000))

In [ ]:
from datetime import datetime
cols = ['text','timestamp']
titles_soup.columns = cols
dates=[]
for i in titles_soup['timestamp']:
    dates.append(dt.datetime.date(i))
titles_soup['dates']=dates

titles_soup.head(10)